In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

In [2]:
from brio.utils.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from pickle import dump, load
import pandas as pd
import numpy as np

from brio.bias.FreqVsFreqBiasDetector import FreqVsFreqBiasDetector

## Importing Data and Trained Classifier

In [3]:
input_data_path = "../data/raw_data/uci-default-of-credit-card/data/data.csv"
local_path_save = '../data/mlflow_artifacts/'

In [4]:
fitted_ohe = load(open(local_path_save + '_ohe.pkl', 'rb')) 
fitted_scaler = load(open(local_path_save + '_scaler.pkl', 'rb'))

/Users/davideposillipo/.pyenv/versions/3.10.9/envs/prova/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/davideposillipo/.pyenv/versions/3.10.9/envs/prova/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
pp = Preprocessing(input_data_path, "default")
X, Y = pp.read_dataframe()

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=420)

X_test_ohe, _, _ = pp.preprocess_for_classification(df=X_test, 
                                                fit_ohe=True, 
                                                fitted_ohe=fitted_ohe,
                                                perform_scaling=True,
                                                fitted_scaler=fitted_scaler)

In [6]:
with open("trained_model_for_testing/RF_12_200.pkl", "rb") as file:
    classifier_strong = load(file)
    
with open("trained_model_for_testing/RF_37_10.pkl", "rb") as file:
    classifier_weak = load(file)
    
with open("trained_model_for_testing/Tree_depth2.pkl", "rb") as file:
    classifier_lame = load(file)

In [7]:
predicted_values_strong = classifier_strong.predict(X_test_ohe)
predicted_values_weak = classifier_weak.predict(X_test_ohe)
predicted_values_lame = classifier_lame.predict(X_test_ohe)

In [8]:
def age_buckets(x):
    if x < 30:
        return 1
    elif x < 40:
        return 2
    else:
        return 3

X_test['age_buckets'] = X.x5_age.apply(age_buckets)

In [9]:
conditioning_variables = ['x3_education', 'x4_marriage', 'age_buckets']

In [10]:
df_with_predictions_strong = pd.concat(
    [X_test.reset_index(drop=True), pd.Series(predicted_values_strong)], axis=1).rename(columns={0:"predictions"})

df_with_predictions_weak = pd.concat(
    [X_test.reset_index(drop=True), pd.Series(predicted_values_weak)], axis=1).rename(columns={0:"predictions"})

df_with_predictions_lame = pd.concat(
    [X_test.reset_index(drop=True), pd.Series(predicted_values_lame)], axis=1).rename(columns={0:"predictions"})

## Experiment 1: freqs-vs-freqs, TVD, A1=low, root_variable=x2_sex

In [11]:
bd = FreqVsFreqBiasDetector(distance="TVD", A1="low")

#### Strong model, no conditioning variables

In [12]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.025269625352224545, True, 0.038868585412256317, None)

#### Weak model, no conditioning variables

In [13]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.02310418899075288, True, 0.038868585412256317, None)

#### Lame model, no conditioning variables

In [14]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.011162043100369085, True, 0.038868585412256317, None)

## Experiment 2: freqs-vs-freqs, TVD, A1=low, root_variable=x3_education

In [15]:
bd = FreqVsFreqBiasDetector(distance="TVD", A1="low")

#### Strong model, no conditioning variables

In [16]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.14609739826551038, False, 0.030917574992439394, 0.04868174081342471)

#### Weak model, no conditioning variables

In [17]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.1428571428571429, False, 0.030917574992439394, 0.04818124491646396)

#### Lame model, no conditioning variables

In [18]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.1428571428571429, False, 0.030917574992439394, 0.04257219130365396)

## Experiment 3: freqs-vs-freqs, JS, A1=low, root_variable=x2_sex

In [19]:
bd = FreqVsFreqBiasDetector(distance="JS", A1="low")

#### Strong model, no conditioning variables

In [20]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.0011441803173238346, True, 0.038868585412256317, None)

#### Weak model, no conditioning variables

In [21]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.0009363117559263138, True, 0.038868585412256317, None)

#### Lame model, no conditioning variables

In [22]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x2_sex'
)

(0.00029060576357160914, True, 0.038868585412256317, None)

## Experiment 4: freqs-vs-freqs, JS, A1=low, root_variable=x3_education

In [23]:
bd = FreqVsFreqBiasDetector(distance="JS", A1="low")

#### Strong model, no conditioning variables

In [24]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.07720553617455106, False, 0.030917574992439394, 0.028868970403301546)

#### Weak model, no conditioning variables

In [25]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.07539593734971191, False, 0.030917574992439394, 0.02886128575319307)

#### Lame model, no conditioning variables

In [26]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x3_education'
)

(0.07539593734971191, False, 0.030917574992439394, 0.024635222244734474)

## Experiment 5: freqs-vs-freqs, JS, A1=low, root_variable=x6_pay_0 (discrete)

In [27]:
def discrete_x6(x):
    if x <= 2:
        return x
    else:
        return 3

df_with_predictions_lame["x6_pay_0_discrete"] = df_with_predictions_lame["x6_pay_0"].apply(lambda x: discrete_x6(x))
df_with_predictions_strong["x6_pay_0_discrete"] = df_with_predictions_strong["x6_pay_0"].apply(lambda x: discrete_x6(x))
df_with_predictions_weak["x6_pay_0_discrete"] = df_with_predictions_weak["x6_pay_0"].apply(lambda x: discrete_x6(x))

In [28]:
bd = FreqVsFreqBiasDetector(distance="JS", A1="low")

#### Strong model, no conditioning variables

In [29]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x6_pay_0_discrete'
)

(0.7711906405718167, False, 0.03144764235376052, 0.33614740604221743)

#### Weak model, no conditioning variables

In [30]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x6_pay_0_discrete'
)

(0.4989003789987942, False, 0.03144764235376052, 0.19953711032941285)

#### Lame model, no conditioning variables

In [31]:
bd.compare_root_variable_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x6_pay_0_discrete'
)

(0.8589211642449359, False, 0.03144764235376052, 0.404899319828159)

## Experiment 6: freqs-vs-freqs, JS, A1=low, root_variable=x2_sex, conditioning_variable=x6_pay (discrete)

In [32]:
bd = FreqVsFreqBiasDetector(distance="JS", A1="low")

#### Strong model

In [33]:
results_strong_ex6 = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [34]:
results_strong_ex6

{'x6_pay_0_discrete==0': (4415,
  3.359847035497652e-05,
  True,
  0.0389193117772519,
  None),
 'x6_pay_0_discrete==2': (788,
  6.792126013614701e-08,
  True,
  0.03915076465618199,
  None),
 'x6_pay_0_discrete==1': (1076,
  0.01079832671100786,
  True,
  0.03909296230297796,
  None),
 'x6_pay_0_discrete==-2': (851, 0.0, True, 0.039135645033232476, None),
 'x6_pay_0_discrete==-1': (1726,
  3.3960511033214674e-05,
  True,
  0.039020789691049496,
  None),
 'x6_pay_0_discrete==3': (144, 0.0004375255127418719, True, 0.0396875, None)}

#### Weak model

In [35]:
results_weak_ex6 = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [36]:
results_weak_ex6

{'x6_pay_0_discrete==0': (4415,
  0.00022767345324620537,
  True,
  0.0389193117772519,
  None),
 'x6_pay_0_discrete==2': (788,
  0.00017464737721858793,
  True,
  0.03915076465618199,
  None),
 'x6_pay_0_discrete==1': (1076,
  0.005624199766508536,
  True,
  0.03909296230297796,
  None),
 'x6_pay_0_discrete==-2': (851,
  0.00556536793974934,
  True,
  0.039135645033232476,
  None),
 'x6_pay_0_discrete==-1': (1726,
  0.0002454704757063513,
  True,
  0.039020789691049496,
  None),
 'x6_pay_0_discrete==3': (144, 0.010663712690594325, True, 0.0396875, None)}

#### Lame model

In [37]:
results_lame_ex6 = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [38]:
results_lame_ex6

{'x6_pay_0_discrete==0': (4415, 0.0, True, 0.0389193117772519, None),
 'x6_pay_0_discrete==2': (788,
  0.00014263243686207786,
  True,
  0.03915076465618199,
  None),
 'x6_pay_0_discrete==1': (1076, 0.0, True, 0.03909296230297796, None),
 'x6_pay_0_discrete==-2': (851, 0.0, True, 0.039135645033232476, None),
 'x6_pay_0_discrete==-1': (1726, 0.0, True, 0.039020789691049496, None),
 'x6_pay_0_discrete==3': (144, 0.0, True, 0.0396875, None)}

## Experiment 7: freqs-vs-freqs, TVD, A1=low, root_variable=x2_sex, conditioning_variable=x6_pay (discrete)

In [39]:
bd = FreqVsFreqBiasDetector(distance="TVD", A1="low")

#### Strong model

In [40]:
results_strong_ex7 = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_strong,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [41]:
results_strong_ex7

{'x6_pay_0_discrete==0': (4415,
  0.0005099035448375608,
  True,
  0.0389193117772519,
  None),
 'x6_pay_0_discrete==2': (788,
  0.00017791835524372246,
  True,
  0.03915076465618199,
  None),
 'x6_pay_0_discrete==1': (1076,
  0.08358658256880735,
  False,
  0.03909296230297796,
  None),
 'x6_pay_0_discrete==-2': (851, 0.0, True, 0.039135645033232476, None),
 'x6_pay_0_discrete==-1': (1726,
  0.001151020781208943,
  True,
  0.039020789691049496,
  None),
 'x6_pay_0_discrete==3': (144, 0.01550688117852296, True, 0.0396875, None)}

#### Weak model

In [42]:
results_weak_ex7 = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_weak,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [43]:
results_weak_ex7

{'x6_pay_0_discrete==0': (4415,
  0.004162196434452259,
  True,
  0.0389193117772519,
  None),
 'x6_pay_0_discrete==2': (788,
  0.013502026292379177,
  True,
  0.03915076465618199,
  None),
 'x6_pay_0_discrete==1': (1076,
  0.07061353211009175,
  False,
  0.03909296230297796,
  None),
 'x6_pay_0_discrete==-2': (851,
  0.030280300039478902,
  True,
  0.039135645033232476,
  None),
 'x6_pay_0_discrete==-1': (1726,
  0.005857924620991439,
  True,
  0.039020789691049496,
  None),
 'x6_pay_0_discrete==3': (144, 0.11145570847063391, False, 0.0396875, None)}

#### Lame model

In [44]:
results_lame_ex7 = bd.compare_root_variable_conditioned_groups(
    dataframe=df_with_predictions_lame,
    target_variable='predictions',
    root_variable='x2_sex',
    conditioning_variables=["x6_pay_0_discrete"],
    min_obs_per_group=30)

In [45]:
results_lame_ex7

{'x6_pay_0_discrete==0': (4415, 0.0, True, 0.0389193117772519, None),
 'x6_pay_0_discrete==2': (788,
  0.005996507528582273,
  True,
  0.03915076465618199,
  None),
 'x6_pay_0_discrete==1': (1076, 0.0, True, 0.03909296230297796, None),
 'x6_pay_0_discrete==-2': (851, 0.0, True, 0.039135645033232476, None),
 'x6_pay_0_discrete==-1': (1726, 0.0, True, 0.039020789691049496, None),
 'x6_pay_0_discrete==3': (144, 0.0, True, 0.0396875, None)}

#### Checking distances ordering for experiment 6 vs 7

In [46]:
#### Checking JS <= TVD

In [47]:
[js[1]<=tvd[1] for js, tvd in zip(results_weak_ex6.values(), results_weak_ex7.values())]

[True, True, True, True, True, True]

In [48]:
[js[1]<=tvd[1] for js, tvd in zip(results_strong_ex6.values(), results_strong_ex7.values())]

[True, True, True, True, True, True]

In [49]:
[js[1]<=tvd[1] for js, tvd in zip(results_lame_ex6.values(), results_lame_ex7.values())]

[True, True, True, True, True, True]

In [50]:
#### Checking JS < TVD

In [51]:
[js[1]<tvd[1] for js, tvd in zip(results_weak_ex6.values(), results_weak_ex7.values())]

[True, True, True, True, True, True]

In [52]:
# The false here is for one case with JS==TVD
[js[1]<tvd[1] for js, tvd in zip(results_strong_ex6.values(), results_strong_ex7.values())]

[True, True, True, False, True, True]

In [53]:
[js[1]<=tvd[1] for js, tvd in zip(results_lame_ex6.values(), results_lame_ex7.values())]

[True, True, True, True, True, True]

###### Focus on the case where the ordering is not respected

In [54]:
# The false here is for one case with JS==TVD
{k: (js[1]<tvd[1], js[1], tvd[1]) for k, js, tvd in zip(results_strong_ex6.keys(), results_strong_ex6.values(), results_strong_ex7.values())}

{'x6_pay_0_discrete==0': (True, 3.359847035497652e-05, 0.0005099035448375608),
 'x6_pay_0_discrete==2': (True, 6.792126013614701e-08, 0.00017791835524372246),
 'x6_pay_0_discrete==1': (True, 0.01079832671100786, 0.08358658256880735),
 'x6_pay_0_discrete==-2': (False, 0.0, 0.0),
 'x6_pay_0_discrete==-1': (True, 3.3960511033214674e-05, 0.001151020781208943),
 'x6_pay_0_discrete==3': (True, 0.0004375255127418719, 0.01550688117852296)}